<a href="https://colab.research.google.com/github/Tanucode/Deep-Learning-Practice/blob/main/Topic%20Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import string
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
df = pd.read_excel ("samplefile1.xlsx",names=['tweet','polarity','frequency'])
df = df.dropna()
print (df)

                                                  tweet polarity  frequency
0     Global warming report urges governments to act...      Yes     1.0000
1     Fighting poverty and global warming in Africa ...      Yes     1.0000
2     Carbon offsets: How a Vatican forest failed to...      Yes     0.8786
3     Carbon offsets: How a Vatican forest failed to...      Yes     1.0000
4     URUGUAY: Tools Needed for Those Most Vulnerabl...      Yes     0.8087
...                                                 ...      ...        ...
6084  It's 83•_Á and climbing in NYC. August weather...        Y     1.0000
6085  @bloodless_coup "The phrase 'global warming' s...        Y     1.0000
6087  Global warming you tube parody you will enjoy ...        N     0.6411
6088  One-Eyed Golfer: Don't dare tell me about glob...        N     1.0000
6089  man made global warming a hair brained theory ...        N     1.0000

[4225 rows x 3 columns]


In [4]:
df = df.drop(columns=['frequency'])
# df_cleaned = df.dropna()
print(df)

                                                  tweet polarity
0     Global warming report urges governments to act...      Yes
1     Fighting poverty and global warming in Africa ...      Yes
2     Carbon offsets: How a Vatican forest failed to...      Yes
3     Carbon offsets: How a Vatican forest failed to...      Yes
4     URUGUAY: Tools Needed for Those Most Vulnerabl...      Yes
...                                                 ...      ...
6084  It's 83•_Á and climbing in NYC. August weather...        Y
6085  @bloodless_coup "The phrase 'global warming' s...        Y
6087  Global warming you tube parody you will enjoy ...        N
6088  One-Eyed Golfer: Don't dare tell me about glob...        N
6089  man made global warming a hair brained theory ...        N

[4225 rows x 2 columns]


In [5]:
df.polarity.value_counts()

Y      2554
N      1053
Yes     557
No       61
Name: polarity, dtype: int64

In [6]:
df['polarity'] = df['polarity'].replace({'Yes': 1, 'Y': 1, 'No': 0, 'N': 0})
print(df)

                                                  tweet  polarity
0     Global warming report urges governments to act...         1
1     Fighting poverty and global warming in Africa ...         1
2     Carbon offsets: How a Vatican forest failed to...         1
3     Carbon offsets: How a Vatican forest failed to...         1
4     URUGUAY: Tools Needed for Those Most Vulnerabl...         1
...                                                 ...       ...
6084  It's 83•_Á and climbing in NYC. August weather...         1
6085  @bloodless_coup "The phrase 'global warming' s...         1
6087  Global warming you tube parody you will enjoy ...         0
6088  One-Eyed Golfer: Don't dare tell me about glob...         0
6089  man made global warming a hair brained theory ...         0

[4225 rows x 2 columns]


In [7]:
df= df.drop_duplicates(subset='tweet')
print(df)

                                                  tweet  polarity
0     Global warming report urges governments to act...         1
1     Fighting poverty and global warming in Africa ...         1
2     Carbon offsets: How a Vatican forest failed to...         1
4     URUGUAY: Tools Needed for Those Most Vulnerabl...         1
5     RT @sejorg: RT @JaymiHeimbuch: Ocean Saltiness...         1
...                                                 ...       ...
6084  It's 83•_Á and climbing in NYC. August weather...         1
6085  @bloodless_coup "The phrase 'global warming' s...         1
6087  Global warming you tube parody you will enjoy ...         0
6088  One-Eyed Golfer: Don't dare tell me about glob...         0
6089  man made global warming a hair brained theory ...         0

[3889 rows x 2 columns]


In [8]:
import re

def clean_tweet(text):
    signs = r'@[^ ]+'                   #"@signs" and values
    links = r'https?://[A-Za-z0-9./]+'  #links
    extraletters = r'\'s'                      #floating s's
    hashtags = r'\#\w+'                     # hashtags and value
    ands = r'&amp '
    nonalphabet = r'[^A-Za-z\s]'         #remove non-alphabet
    combined_words = r'|'.join((signs, links,extraletters,hashtags,ands, nonalphabet))
    text = re.sub(combined_words,"",text).lower()
    return text.strip()

In [9]:
df["cleaned_tweet"]=df["tweet"].apply(clean_tweet)
print(df)

                                                  tweet  polarity  \
0     Global warming report urges governments to act...         1   
1     Fighting poverty and global warming in Africa ...         1   
2     Carbon offsets: How a Vatican forest failed to...         1   
4     URUGUAY: Tools Needed for Those Most Vulnerabl...         1   
5     RT @sejorg: RT @JaymiHeimbuch: Ocean Saltiness...         1   
...                                                 ...       ...   
6084  It's 83•_Á and climbing in NYC. August weather...         1   
6085  @bloodless_coup "The phrase 'global warming' s...         1   
6087  Global warming you tube parody you will enjoy ...         0   
6088  One-Eyed Golfer: Don't dare tell me about glob...         0   
6089  man made global warming a hair brained theory ...         0   

                                          cleaned_tweet  
0     global warming report urges governments to act...  
1     fighting poverty and global warming in africa ...

In [10]:
df = df.drop(columns=['tweet'])
print(df)

      polarity                                      cleaned_tweet
0            1  global warming report urges governments to act...
1            1  fighting poverty and global warming in africa ...
2            1  carbon offsets how a vatican forest failed to ...
4            1  uruguay tools needed for those most vulnerable...
5            1  rt  rt  ocean saltiness shows global warming i...
...        ...                                                ...
6084         1  it  and climbing in nyc august weather for the...
6085         1  the phrase global warming should be abandoned ...
6087         0      global warming you tube parody you will enjoy
6088         0  oneeyed golfer dont dare tell me about global ...
6089         0  man made global warming a hair brained theory ...

[3889 rows x 2 columns]


In [11]:
df.polarity.value_counts()

1    2845
0    1044
Name: polarity, dtype: int64

In [12]:
count_nan = df.isna().sum().sum()
print(count_nan)

0


In [13]:
df['cleaned_tweet'].str.lower()
split_data = df["cleaned_tweet"].str.split(" ")
data = split_data
print(df)

      polarity                                      cleaned_tweet
0            1  global warming report urges governments to act...
1            1  fighting poverty and global warming in africa ...
2            1  carbon offsets how a vatican forest failed to ...
4            1  uruguay tools needed for those most vulnerable...
5            1  rt  rt  ocean saltiness shows global warming i...
...        ...                                                ...
6084         1  it  and climbing in nyc august weather for the...
6085         1  the phrase global warming should be abandoned ...
6087         0      global warming you tube parody you will enjoy
6088         0  oneeyed golfer dont dare tell me about global ...
6089         0  man made global warming a hair brained theory ...

[3889 rows x 2 columns]


In [14]:
# Assuming df is your DataFrame containing the 'cleaned_tweet' column
keywords_global_warming = ['global warming', 'climate change', 'carbon emissions', 'environment']

# Function to check if any keyword appears in the cleaned_tweet
def detect_global_warming_topic(tweet):
    for keyword in keywords_global_warming:
        if keyword in tweet.lower():  # Convert both the tweet and the keyword to lowercase for case-insensitive matching
            return True
    return False

# Apply the function to the DataFrame
df['is_global_warming_topic'] = df['cleaned_tweet'].apply(detect_global_warming_topic)

# Filter rows where the global warming topic is detected
global_warming_tweets = df[df['is_global_warming_topic']]

# Print the detected global warming tweets
print(global_warming_tweets[['cleaned_tweet', 'polarity']])

                                          cleaned_tweet  polarity
0     global warming report urges governments to act...         1
1     fighting poverty and global warming in africa ...         1
2     carbon offsets how a vatican forest failed to ...         1
4     uruguay tools needed for those most vulnerable...         1
5     rt  rt  ocean saltiness shows global warming i...         1
...                                                 ...       ...
6084  it  and climbing in nyc august weather for the...         1
6085  the phrase global warming should be abandoned ...         1
6087      global warming you tube parody you will enjoy         0
6088  oneeyed golfer dont dare tell me about global ...         0
6089  man made global warming a hair brained theory ...         0

[3694 rows x 2 columns]


In [22]:
# Assuming df is your DataFrame containing the 'cleaned_tweet' column
keywords_global_warming = ['global warming', 'carbon emissions']
keywords_climate_change = ['climate change', 'environment']
keyword_poverty = ['poverty']
keyword_energy = ['renewable energy', 'energy consumption']

# Function to check if any keyword appears in the cleaned_tweet
def detect_global_warming_topic(tweet):
    for keyword in keywords_global_warming:
        if keyword in tweet.lower():  # Convert both the tweet and the keyword to lowercase for case-insensitive matching
            return True
    return False

def detect_climate_change_topic(tweet):
    for keyword in keywords_climate_change:
        if keyword in tweet.lower():  # Convert both the tweet and the keyword to lowercase for case-insensitive matching
            return True
    return False


def detect_poverty_topic(tweet):
    for keyword in keyword_poverty:
        if keyword in tweet.lower():  # Convert both the tweet and the keyword to lowercase for case-insensitive matching
            return True
    return False

def detect_energy_topic(tweet):
    for keyword in keyword_energy:
        if keyword in tweet.lower():  # Convert both the tweet and the keyword to lowercase for case-insensitive matching
            return True
    return False

# Apply the function to the DataFrame
df['is_global_warming_topic'] = df['cleaned_tweet'].apply(detect_global_warming_topic)
df['is_climate_change_topic'] = df['cleaned_tweet'].apply(detect_climate_change_topic)
df['is_poverty_topic'] = df['cleaned_tweet'].apply(detect_poverty_topic)
df['is_energy_topic'] = df['cleaned_tweet'].apply(detect_energy_topic)

# Filter rows where the global warming topic is detected
global_warming_tweets = df[df['is_global_warming_topic']]

# Filter rows where the climate change topic is detected
climate_change_tweets = df[df['is_climate_change_topic']]

#Filter rows where poverty as a topic is detected
poverty_tweets = df[df['is_poverty_topic']]

#Filter rows where energy as a topic is detected
energy_tweets = df[df['is_energy_topic']]

# Print the detected global warming tweets
print(global_warming_tweets[['cleaned_tweet', 'polarity']])

# Print the detected climate change tweets
print(climate_change_tweets[['cleaned_tweet', 'polarity']])

#print the detected poverty tweets
print(poverty_tweets[['cleaned_tweet','polarity']])

#print the detected energy tweets
print(energy_tweets[['cleaned_tweet','polarity']])


                                          cleaned_tweet  polarity
0     global warming report urges governments to act...         1
1     fighting poverty and global warming in africa ...         1
2     carbon offsets how a vatican forest failed to ...         1
5     rt  rt  ocean saltiness shows global warming i...         1
6     global warming evidence all around usa message...         1
...                                                 ...       ...
6084  it  and climbing in nyc august weather for the...         1
6085  the phrase global warming should be abandoned ...         1
6087      global warming you tube parody you will enjoy         0
6088  oneeyed golfer dont dare tell me about global ...         0
6089  man made global warming a hair brained theory ...         0

[2014 rows x 2 columns]
                                          cleaned_tweet  polarity
4     uruguay tools needed for those most vulnerable...         1
7     migratory birds new climate change strategy s